## Jrfapp tutorial 2 
## Inverting a synthetic Model using a real dataset with noise:

First we need to import Initialize and Jrfapp_station. You can perform it by:

In [2]:
from jrfapp.main_classes import Initialize
from jrfapp.main_classes import Jrfapp_station
import os

In this run we want to read a real dataset from a station and perform a synthetic with 50.0 % noise level. 
The first step is to initialize the code. However, in this run we need to define the folder of our netwwork. This folder 
can contain several station data. Each station data is in a folder with a name according to an input file which define 
the station name and coordinates of all station of corresponding network. In this example i used the dataset of a permanent
station in the CB network. We need to give the coordinates file name of this network in the Initialize class. The Initialize
class is then defined as below and you must change these path according to your saved data/path. 

In [ ]:
#%%
### Folders that need to be defined:

## Path to the data folder. This folder must contain your stations folder data. 
data_folder = '/mnt/4TB/soroush/CB_network'
###############################################################################
## Path to the folder which contains references models. The folder must only 
## contain .dat files. 
model_folder = '/home/soroush/my_py_rf/model_folder'
###############################################################################
## Path to the specific refrence initial model. These option used because you may want to
## use different model as the reference model. 
model_name='/home/soroush/my_py_rf/model_folder/continent_tibet_model.dat'
###############################################################################
## Path to the coordinates file. This file must contain station name, lat and 
## long of the stations. Note that the station name in this file, the station 
## name in the data_folder and station name given to thr Jrfapp_station class 
## must be same.
station_coordinate_file= '/home/soroush/my_py_rf/CB_coordinates'
###############################################################################
## The folder which package use for saving the output of stations. 
output_folder='/mnt/4TB/soroush/CB_st/syn_st'

In [ ]:
init_obj = Initialize(network_name='CB', 
                               station_coordinate_file= station_coordinate_file,
                               data_folder=data_folder, 
                               layering= [3, 4, 3], 
                  output_folder=output_folder, 
                   model_name= 'halfspace',
                   random_seed= 250)

The CB_coordinates file contains the station name, latitude and longitude. 
for example 
> CAD_P     31.0    97.50

is one of the lines in CB_coordinates that correspond to CAD_P station.

The data_folder is where we saved the dataset of CAD_P station. the data_folder structures must be as follow:
> {data_folder}/{station_folder}/{event_folder}/*BHZ.SAC *BHE.SAC *BHN.SAC
for example:
> data_folder/CAD_P/2013031033343/CAD_2013031033343_BHE.SAC,
> data_folder/CAD_P/2013031033343/CAD_2013031033343_BHN.SAC,  
> data_folder/CAD_P/2013031033343/CAD_2013031033343_BHZ.SAC.



Now we need to define our synthetic model and Jrfapp_stobj for this station as follows:

In [ ]:
which_to_pert = []
which_to_pert.append([6, 12,  -0.6])
which_to_pert.append([18, 24,  0.6])
which_to_pert.append([30, 36, -0.6])
which_to_pert.append([50, 60,  0.6])
init_obj.create_synthetic(which_to_pert)
jrfapp_stobj = Jrfapp_station(init_obj, name = 'CAD_P', 
                                      noise_level= 50.0)

Next we can invert the data by either RMS or PSO method as the pseudo-initial model estimator. But first lets save the Jrfapp_stobj as the processes
for calculating RF and Apparent velocity and stacking them can take long time and we dont want to repeat this processes 
every time. 

In [ ]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='syn_with_noise_gs_bf_inv.bin')

In [ ]:
we can later load this data by pickle.

In [ ]:
import pickle
with open(jrfapp_stobj_file_name, 'rb') as f1:
    jrfapp_stobj = pickle.load(f1)

Now we can invert synthetic data by:

In [ ]:
jrfapp_stobj.invert_data(inv_method = 'grid_search',
                     stack_name = 'synthetic',
                     ndivide_list = [-1, 1, -2, 2, -3, 3, -4, 4], 
                     nmodel = 6, nthread = 6, 
                     finer_ndivide= [-2, 2, -3, 3, -4, 4, -5, 5])

In [ ]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='syn_with_noise_gs_af_inv.bin')

Inversion using PSO:

In [ ]:
jrfapp_stobj.invert_data(inv_method = 'PSO',
                stack_name = 'synthetic',
                ndivide_list = [-1, 1, -2, 2, -3, 3, -4, 4], 
                PSO_nparticle = 4, PSO_maxiter = 2, 
                nthread = 6)

<div class="alert alert-block alert-info">
<b>Tip:</b> Again for simplicity I run the inversion with 6 models for grid_search and 4 particles in 2 iterations for the PSO run.
</div>

In [ ]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='syn_with_noise_pso_af_inv.bin')